# Spatial Narratives Project

#### **Generating the working dataset from the CLDW**

In [ ]:
!git clone https://github.com/SpaceTimeNarratives/demo.git

Cloning into 'demo'...
remote: Enumerating objects: 341, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 341 (delta 5), reused 11 (delta 3), pack-reused 324
Receiving objects: 100% (341/341), 35.52 MiB | 28.26 MiB/s, done.
Resolving deltas: 100% (148/148), done.


In [ ]:
cd demo

/content/demo


In [ ]:
import os
import string
import re
import spacy
import pandas as pd
from bs4 import BeautifulSoup
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
stop_words = stopwords.words('english')
lemma = WordNetLemmatizer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

### Setting up the extraction pipeline

In [ ]:
pip -q install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 917.6/917.6 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 42.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 769.7/769.7 kB 50.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 668.8/668.8 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 75.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.9/51.9 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.5/97.5 kB 11.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
inflect 6.0.4 requires pydantic>=1.9.1, but you have pydantic 1.8.2 which is incompatible.
en-core-web-sm 3.5.0 requires spacy<3.6.0,>=3.5.0, but you have spacy 3.3.1 which is incompatible.


In [ ]:
%run functions.py

In [ ]:
# Get the list of placenames and geonouns
place_names = [name.strip().title().replace("'S", "'s") for name in open('LD_placenames.txt').readlines()] #read and convert to title case 
place_names += [name.upper() for name in place_names] #retain the upper case versions
geonouns = get_inflections([noun.strip() for noun in open('geo_feature_nouns.txt').readlines()])

# Get the list of positive and negative words from the sentiment lexicon
pos_words = [w.strip() for w in open('positive-words.txt','r', encoding='latin-1').readlines()[35:]]
neg_words = [w.strip() for w in open('negative-words.txt','r', encoding='latin-1').readlines()[35:]]

# Create a blank spacy English model
nlp = spacy.blank("en")
ruler = nlp.add_pipe("entity_ruler")

# Define the patterns for the EntityRuler by labelling all the names with the tag PLNAME
patterns = [{"label": "PLNAME", "pattern": plname} for plname in set(place_names)]
patterns += [{"label": "GEONOUN", "pattern": noun} for noun in geonouns]
patterns += [{"label": "+EMOTION", "pattern": word} for word in pos_words]
patterns += [{"label": "-EMOTION", "pattern": word} for word in neg_words]

ruler.add_patterns(patterns)

## Extract and tag the paragraphs

In [ ]:
from spacy import displacy
options = {'colors':BG_COLOR}

Define the `get_paragraph()` function that cleans the text and returns a list of text paragraphs

In [ ]:
def get_paragraphs(input_text):
  paragraphs = []
  soup = BeautifulSoup(input_text, 'html.parser')
  
  # Define a regular expression pattern to match XML tags
  pattern = re.compile(r'<.*?>')

  for i, p in enumerate(soup.find_all('p')):
    # Use the sub() function to remove all tags from the XML text
    _text = re.sub(pattern, '', str(p))

    # use the nltk sentence tokenizer to segment the text into sentences
    _text = _text.replace('\n', ' '
              ).replace('\t', ' '
                  ).replace('∫', 's'
                      ).replace("\'", "'")

    # Replace multiple spaces with one space
    paragraphs.append(re.sub(r'\s+', ' ', _text))

  # Split into sentences, strip leading and trailing non-printables and return 
  return paragraphs

Build the `data` dictionary by
1. Reading all files in the `gold_standard\` folder
2. Applying the `get_paragraphs()` function to return the paragraphs in each file.
3. For each paragragh, store the `paraId`, `text`,  and `word_count`

In [ ]:
paragraphs = []
for fileId, filename in enumerate(sorted(os.listdir('gold_standard'))):
  text = open(f'gold_standard/{filename}', 'r', encoding='utf8').read()
  for paraId, paragraph in enumerate(get_paragraphs(text)):
    paragraphs.append({'fileId':fileId,'paraId':paraId, 'text':paragraph, 'word_count':len(paragraph.split())})

Convert it to a Pandas dataframe for viewing

In [ ]:
data = pd.DataFrame.from_dict(paragraphs)
data

,fileId,paraId,text,word_count
0,0,0,By the route which we have traced among the En...,112
1,0,1,With the carefully prepared map attached to th...,631
2,0,2,At all seasons the scenery of the lake distric...,68
3,0,3,This is a clean-looking ancient town in the so...,122
4,0,4,"From Penrith two roads lead to Pooley Bridge, ...",89
...,...,...,...,...
1739,27,179,The following sketches of farms will give an i...,13
1740,27,180,"Another,",1
1741,27,181,"Another,",1
1742,27,182,"Another,",1


Update the `data` dictionary. For each file
1. Go through all the paragraphs
2. Extract all PLNAMES and GEONOUNS. +/-EMOTIONS
3. Store the list of each tags as well as the counts

In [ ]:
def pre_process_text(text):
  return list(filter(lambda token: token not in string.punctuation,
             [lemma.lemmatize(word) for word in word_tokenize(text) 
             if word.lower() not in stop_words]))
# Get entity counts for each tag
def get_entities(text, tag):
  return [(ent, ent.start_char, ent.end_char) for ent in nlp(text).ents if ent.label_ == tag]
  
def add_entity_count(data_df, tag):
  ents = [get_entities(text, tag) for text in data_df['text']]
  counts = [len(count) for count in ents]
  return ents, counts

data['plnames'], data['pn_cnts'] = add_entity_count(data, 'PLNAME')
data['geonouns'], data['gn_cnts'] = add_entity_count(data, 'GEONOUN')
data['pos_emotions'], data['pos_cnts'] = add_entity_count(data, '+EMOTION')
data['neg_emotions'], data['neg_cnts'] = add_entity_count(data, '-EMOTION')
data['sentiment_score'] = (data['pos_cnts'] - data['neg_cnts'])/data['text'].apply(lambda x : len(pre_process_text(x)))

List all occurences of entities (place names, geonouns, emotions) with their file and paragraph IDs

In [ ]:
# Define the `add_tag` function to attach the tag to each entity from a given list
add_tag = lambda x_list, tag: [(x,tag) for x in x_list]

In [ ]:
entities_df = data[['plnames','geonouns', 'pos_emotions', 'neg_emotions','fileId', 'paraId']]
ent_list = []
for i in range(len(entities_df)): 
  plns, gns, pos, neg, fId, pId = entities_df.iloc[i]
  ents = add_tag(plns,'PLNAME')+add_tag(gns,'GEONOUN')+add_tag(pos,'+EMOTION'
                                                  ) + add_tag(neg, '-EMOTION')
  if ents:
    for ent, tag in ents:
      ent_list.append({'entity': ent[0], 'start_char':ent[1], 'end_char':ent[2], 
                      'fileId':fId, 'paraId':pId, 'tag':tag})
ent_list_df = pd.DataFrame.from_dict(ent_list)
ent_list_df

Convert it to Excel for export

In [ ]:
with pd.ExcelWriter('paragraph_counts_v0.xlsx') as writer:  
    data.to_excel(writer, sheet_name='paragraphs')
    ent_list_df.to_excel(writer, sheet_name='all entities')